<a href="https://colab.research.google.com/github/Yaswanthyarra/GAN-for-microexpression-augmentation/blob/main/GAN_Optimised_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
dataset_dir = '/content/drive/My Drive/CASMEII/Final_Apex_Frames_Segregated'
augmented_dir='/content/drive/My Drive/CASMEII/Augmented_Apex_Frames'

In [44]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np

import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split


dataset_dir = '/content/drive/My Drive/CASMEII/Final_Apex_Frames_Segregated'

label_dict = {'happiness': 0, 'disgust': 1, 'repression': 2, 'surprise': 3, 'fear': 4, 'others': 6, 'sadness': 5}

expression_images = []
expression_labels = []

for class_name, label in label_dict.items():
    class_dir = os.path.join(dataset_dir, class_name)
    images = os.listdir(class_dir)

    for image_file in images:
        image_path = os.path.join(class_dir, image_file)
        image = cv2.imread(image_path)  # Load the image using OpenCV
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (64,64))
        if label==1:
          expression_images.append(image)
          expression_labels.append(label)
        if label==6:
          expression_images.append(image)
          expression_labels.append(label)



        expression_images.append(image)
        expression_labels.append(label)


In [46]:
augmented_dir='/content/drive/My Drive/CASMEII/Augmented_Apex_Frames'
label_dict = {'happiness': 0, 'repression': 2, 'surprise': 3, 'fear': 4, 'sadness': 5}

for class_name, label in label_dict.items():
    class_dir = os.path.join(augmented_dir, class_name)
    images = os.listdir(class_dir)

    for image_file in images:
        image_path = os.path.join(class_dir, image_file)
        image = cv2.imread(image_path)  # Load the image using OpenCV
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        if label==0:
          expression_images.append(image)
          expression_labels.append(label)

        expression_images.append(image)
        expression_labels.append(label)

In [47]:

# Convert lists to NumPy arrays
expression_images = np.array(expression_images)
expression_labels = np.array(expression_labels)

# Check the shape of the image data and labels
print("Expression Images Shape:", expression_images.shape)
print("Expression Labels Shape:", expression_labels.shape)
# Obtain the number of class labels
num_classes = len(np.unique(expression_labels))

# Stratified Splitting of data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(expression_images, expression_labels, test_size=0.2, stratify=expression_labels, random_state=42)

Expression Images Shape: (1147, 64, 64)
Expression Labels Shape: (1147,)


In [52]:
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the model architecture
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.29),
    Dense(num_classes, activation='softmax')
])

# Define the optimizer with a specific learning rate
optimizer = Adam(learning_rate=0.0006)

# Compile the model
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [53]:

# Train the model
model.fit(train_images, train_labels, epochs=52, batch_size=17, validation_data=(test_images, test_labels))


Epoch 1/52
54/54 [==============================] - 7s 118ms/step - loss: 4.0883 - accuracy: 0.3250 - val_loss: 1.1582 - val_accuracy: 0.7217
Epoch 2/52
54/54 [==============================] - 8s 157ms/step - loss: 0.9292 - accuracy: 0.7077 - val_loss: 0.5542 - val_accuracy: 0.8348
Epoch 3/52
54/54 [==============================] - 6s 119ms/step - loss: 0.5463 - accuracy: 0.8266 - val_loss: 0.4871 - val_accuracy: 0.8348
Epoch 4/52
54/54 [==============================] - 8s 152ms/step - loss: 0.4466 - accuracy: 0.8408 - val_loss: 0.4129 - val_accuracy: 0.8696
Epoch 5/52
54/54 [==============================] - 6s 119ms/step - loss: 0.3915 - accuracy: 0.8593 - val_loss: 0.4266 - val_accuracy: 0.8783
Epoch 6/52
54/54 [==============================] - 10s 193ms/step - loss: 0.3880 - accuracy: 0.8582 - val_loss: 0.4099 - val_accuracy: 0.8652
Epoch 7/52
54/54 [==============================] - 6s 120ms/step - loss: 0.3330 - accuracy: 0.8659 - val_loss: 0.3668 - val_accuracy: 0.8783
Epoch

In [55]:
model.save("/content/drive/My Drive/CASMEII/GAN_CNN_Modal.h5")

In [56]:

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('Test accuracy:', test_acc)

8/8 - 0s - loss: 0.3470 - accuracy: 0.9217 - 344ms/epoch - 43ms/step
Test accuracy: 0.9217391014099121


In [58]:
from sklearn.metrics import classification_report
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)



# Calculate precision, recall, F1-score, and support
report = classification_report(test_labels, predicted_labels)
print(report)

8/8 [==============================] - 1s 89ms/step
              precision    recall  f1-score   support

           0       0.89      0.95      0.92        58
           1       0.81      0.84      0.82        25
           2       1.00      0.94      0.97        31
           3       0.88      0.88      0.88        25
           4       1.00      1.00      1.00        24
           5       1.00      0.93      0.96        27
           6       0.92      0.90      0.91        40

    accuracy                           0.92       230
   macro avg       0.93      0.92      0.92       230
weighted avg       0.92      0.92      0.92       230

